**Setup the application**

Import required libraries

In [1]:
import glob, sys, threading

from omegaconf import OmegaConf
from tqdm.auto import tqdm
from grams.prelude import GRAMS, ROOT_DIR, DATA_DIR, I, fetch_tables, WikidataSemanticModelHelper

from sm_widgets.widgets.annotator import Annotator, BatchAnnotator
from sm_widgets_integration.wikidata import get_qnode_db, get_ontclass_db, get_ontprop_db
from sm_widgets_integration.with_grams import convert_table
from sm_widgets.widgets.slider import DisplayShell, Slider
from sm.prelude import M

Create GRAMS instance. `DATA_DIR` is where GRAMS will store entities retrieved from Wikidata, `cfg` contains configuration of GRAMS

In [2]:
cwd = ROOT_DIR / "examples/misc"
cfg = OmegaConf.load(ROOT_DIR / "grams.yaml")
grams = GRAMS(DATA_DIR, cfg, proxy=True)

**Fetch tables from a webpage and select table that we want to annotate**

Fetch table from a Wikipedia page

In [8]:
tables = fetch_tables("https://en.wikipedia.org/wiki/President_of_the_National_Council_(Austria)")
# tables = fetch_tables("https://en.wikipedia.org/wiki/Novartis")

Create a slider to view fetch tables and select the table we want to annotate

In [9]:
slider = Slider(app=DisplayShell(render_fn=lambda df: display(df)), app_render_fn='set_data', embedded_error_log=True)
slider.render()

threading.Thread(
    target=lambda: slider.set_data([
        {"description": "<br>" + " > ".join([r.header for r in table.context]), "args": [table.as_df()]}
        for table in tables
    ])
).start()

HTML(value='<div id="e6a74cdd-d7bc-42fc-9883-e7c06fe1064b"></div>')

Output()

<IPython.core.display.Javascript object>

SlowTunnelWidget(js_endpoint=(0, ''), py_endpoint=(0, ''), tunnel_id='fbd1c876-f810-49c9-893f-07afd07add5f')

Output()

<IPython.core.display.Javascript object>

**Annotate the selected table**

To annotate a table, simply call `grams.annotate`

In [5]:
selected_table = 10
table = tables[selected_table]
table = table.as_relational_linked_table(table_id=f"{table.page_url}/{selected_table}")
annotation = grams.annotate(table)

save table data (optional)

**Display and edit the annotation**

Run the following two cells to render an interactive widget that allows to curate the predicted annotation. The former cell renders the widget, and the later cell sends data to the widget.

In [6]:
annotator = Annotator(
    entities=get_qnode_db(grams.qnodes), ontclasses=get_ontclass_db(grams.wdclasses), ontprops=get_ontprop_db(grams.wdprops), 
    prop_type_uri='http://www.wikidata.org/prop/P31', savedir=cwd / "ground-truth", eshost='http://mira.isi.edu:9200', dev=False, assistant=None)
annotator.render(True)

SlowTunnelWidget(js_endpoint=(0, ''), py_endpoint=(0, ''), tunnel_id='8531e7a7-f23c-4951-8ae3-d51375c0a086')

HTML(value='<div id="11927c6b-20c4-4997-997c-0d7fb073880a"></div>')

<IPython.core.display.Javascript object>

In [7]:
threading.Thread(
    target=lambda: annotator.set_data(table.get_friendly_fs_id(), convert_table(table), sms=[annotation.sm])
).start()